# Store manager application (Week 2)
- This project was inspired from an idea to make erp systems quite easy to use
- This allows store keeper add, edit and even delete products from their inventory through a chat interface
- A amazing effect i noticed from using llms was how calculations like total stock value could be made even though these weren't implemented as functions
- A possible improvment i'll like to work on would be including vlm, allowing users to add products through pictures

In [ ]:
from openai import OpenAI
import gradio as gr
import dotenv
import sqlite3
import json

In [ ]:
dotenv.load_dotenv()

openai = OpenAI()
ollama = OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")
MODEL="gpt-4o-mini"


In [ ]:
#setup sqllite database, create a table of the following columns: id, name, cost_price, selling_price, quantity, image which is optional

DB = "store_keeper.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS products (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, cost_price REAL, selling_price REAL, quantity INTEGER, image TEXT)")
    conn.commit()


In [ ]:
#create a function to add a new product to the table

def add_product(name, cost_price, selling_price, quantity, image=None):
    conn = sqlite3.connect(DB)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO products (name, cost_price, selling_price, quantity, image) VALUES (?, ?, ?, ?, ?)", (name, cost_price, selling_price, quantity, image))
    conn.commit()
    return "Product added successfully"

In [ ]:
#create a function to update a product in the table

def update_product(id, name=None, cost_price=None, selling_price=None, quantity=None, image=None):
    # Only update fields that were explicitly passed (not None)
    updates = []
    params = []
    if name is not None:
        updates.append("name=?")
        params.append(name)
    if cost_price is not None:
        updates.append("cost_price=?")
        params.append(cost_price)
    if selling_price is not None:
        updates.append("selling_price=?")
        params.append(selling_price)
    if quantity is not None:
        updates.append("quantity=?")
        params.append(quantity)
    if image is not None:
        updates.append("image=?")
        params.append(image)
    if not updates:
        return "No fields to update"
    params.append(id)
    conn = sqlite3.connect(DB)
    cursor = conn.cursor()
    cursor.execute(f"UPDATE products SET {', '.join(updates)} WHERE id=?", params)
    conn.commit()
    return "Product updated successfully"

In [ ]:
#create a function to delete a product from the table

def delete_product(id):
    conn = sqlite3.connect(DB)
    cursor = conn.cursor()
    print("id", id)
    cursor.execute("DELETE FROM products WHERE id=?", (id,))
    conn.commit()
    return "Product deleted successfully"

In [ ]:
#create a function to get all products from the table, 
#id can be passed which means it returns a single product, 
#if not, it returns all products, it should also have search functionality

def get_products(id=None, search=None):
    conn = sqlite3.connect(DB)
    cursor = conn.cursor()
    if id:
        cursor.execute("SELECT * FROM products WHERE id=?", (id,))
    elif search:
        cursor.execute("SELECT * FROM products WHERE name LIKE ?", (f"%{search}%",))
    else:
        cursor.execute("SELECT * FROM products")
    products = cursor.fetchall()
    return products


In [ ]:
add_product_function = {
    "name": "add_product",
    "description": "Add a new product to the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the product",
            },
            "cost_price": {
                "type": "number",
                "description": "The cost price of the product",
            },
            "selling_price": {
                "type": "number",
                "description": "The selling price of the product",
            },
            "quantity": {
                "type": "number",
                "description": "The quantity of the product",
            },
            "image": {
                "type": "string",
                "description": "The image of the product",
            },
        },  
        "required": ["name", "cost_price", "selling_price", "quantity"],
        "additionalProperties": False
    }
}

In [ ]:
update_product_function = {
    "name": "update_product",
    "description": "Update a product in the database.",
    "parameters": {
        "type": "object",
        "properties": {
             "id": {
                "type": "number",
                "description": "The id of the product",
            },
            "name": {
                "type": "string",
                "description": "The name of the product",
            },
            "cost_price": {
                "type": "number",
                "description": "The cost price of the product",
            },
            "selling_price": {
                "type": "number",
                "description": "The selling price of the product",
            },
            "quantity": {
                "type": "number",
                "description": "The quantity of the product",
            },
            "image": {
                "type": "string",
                "description": "The image of the product",
            },
        },  
        "required": ["id"],
        "additionalProperties": False
    }
}

In [ ]:
delete_product_function = {
    "name": "delete_product",
    "description": "Delete a product from the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "id": {
                "type": "number",
                "description": "The id of the product",
            },
        },  
        "required": ["id"],
        "additionalProperties": False
    }
}

In [ ]:
get_products_function = {
    "name": "get_products",
    "description": "Get all products from the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "id": {
                "type": "number",
                "description": "The id of the product",
            },
            "search": {
                "type": "string",
                "description": "The search query",
            },
        },  
        "required": [],
        "additionalProperties": False
    }
}

In [ ]:
tools = [
    {"type": "function", "function": add_product_function}, 
    {"type": "function", "function": update_product_function}, 
    {"type": "function", "function": delete_product_function}, 
    {"type": "function", "function": get_products_function}]
tools

In [ ]:
function_mapping = {
    "add_product": add_product,
    "update_product": update_product,
    "delete_product": delete_product,
    "get_products": get_products,
}




In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        if function_name not in function_mapping:
            response = "Tool not found"
        else:
            arguments = json.loads(tool_call.function.arguments)
            response = function_mapping[function_name](**arguments)
        # Tool response content must be a string for the API
        content = response if isinstance(response, str) else json.dumps(response)
        responses.append({
            "role": "tool",
            "content": content,
            "tool_call_id": tool_call.id
        })
    return responses

In [ ]:
system_prompt = """
You are opeyemi a store keeper for luvable baby care stores. You can add, update, delete and get products from the database. you are polite and friendly and ask for permission before proceeding with the task, if you want to delete a product, you have to fetch all product, then delete each
"""

def chat_with_store_keeper(message, history):
    # print("message", message)
    # print("history", history)
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model="gpt-oss", messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = ollama.chat.completions.create(model="gpt-oss", messages=messages, tools=tools)

    # Gradio expects the assistant reply; never return None
    content = response.choices[0].message.content
    return content if content is not None else "Done."

In [ ]:
gr.ChatInterface(chat_with_store_keeper, title="Store Keeper", type="messages").launch(inbrowser=True)